In [1]:
library(conflicted)
library(readxl)
library(gtools)
library(plm)
library(tidyverse)
library(modelsummary)
library(huxtable)
library(dplyr)
library(writexl)
library(sjPlot)
library(DescTools)
library(caTools)
library(psych)
library(pcse)
library(stringr)
conflict_prefer("lag", "stats")
conflict_prefer("Mean", "modelsummary")
conflict_prefer("N", "modelsummary")
conflict_prefer("SD", "modelsummary")
conflicted::conflicts_prefer(plm::lag)

# Override default glance.pgmm for summary control
glance.pgmm <- function(x, robust = FALSE, ...) {
  asummary <- summary(x, robust = robust)
  pdim <- attr(asummary, "pdim")
  data.frame(
    N      = pdim$nT$n * (pdim$nT$T - 1),
    Sargan = asummary$sargan$p.value[["chisq"]],
    AR1    = asummary$m1$p.value[1],
    AR2    = asummary$m2$p.value[1]
  )
}
Path.HOME <- path.expand("~")


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
[conflicted] Will prefer stats::lag over any other package.
[conflicted] Will prefer modelsummary::Mean over any other package.
[conflicted] Will prefer modelsummary::N over any other package.
[conflicted] Will prefer modelsummary::SD over any other package.
[conflicted] Removing existing preference.
[conflicted] Will prefer plm::lag over any other package.


In [2]:
gm <- tibble::tribble(
  ~raw, ~clean, ~fmt,
  "N", "N", "%d",
  "Sargan", "Sargan", "%.4f",
  "AR1", "AR(1)", "%.4f",
  "AR2", "AR(2)", "%.4f",
  "nobs", "N", "%d",
  "time_effect", "时间效应", "%s",
  "twoways_effect", "个体效应", "%s",
  "utest_top", "U-test 拐点", "%s",
  "utest_x_max", "U-test 上限", "%.4f",
  "utest_x_min", "U-test 下限", "%.4f"
)

SIG <- c("*" = .1, "**" = .05, "***" = .01)

names <- c(
  "SDI" = "SDI",
  "SDI_Average" = "SDI_Average",
  "SDI_Equal" = "SDI_Equal",
  "lnFDIstock" = "lnFDIstock",
  "lnFDIstock_w" = "lnFDIstock",
  "lnFDIstock_w2" = "lnFDIstock",
  "I(lnFDIstock^2)" = "lnFDIstock^2",
  "I(lnFDIstock_w^2)" = "lnFDIstock^2",
  "I(lnFDIstock_w2^2)" = "lnFDIstock^2",
  "s_lnFDIstock_w" = "lnFDIstock",
  "low_income" = "low_income",
  "uppermiddle_income" = "uppermiddle_income",
  "I(s_lnFDIstock_w * low_income)" = "lnFDIstock*low_income",
  "I(s_lnFDIstock_w * uppermiddle_income)" = "lnFDIstock*uppermiddle_income",
  "oil_country " = "oil_country ",
  "I(s_lnFDIstock_w * oil_country)" = "lnFDIstock*oil_country",
  "mineral_country" = "mineral_country",
  "I(s_lnFDIstock_w * mineral_country)" = "lnFDIstock*mineral_country",
  "financial" = "financial",
  "urban" = "urban",
  "WGI" = "WGI",
  "rqe" = "rqe",
  "rle" = "rle",
  "cce" = "cce",
  "pve" = "pve",
  "gee" = "gee",
  "vae" = "vae",
  "property" = "property",
  "GFC" = "GFC",
  "tax" = "tax",
  "war_intensity" = "war_intensity",
  "lag(financial)" = "l.financial",
  "lag(urban)" = "l.urban",
  "lag(property" = "l.property",
  "lag(tax)" = "l.tax",
  "lag(war_intensity)" = "l.war_intensity"
)

utest <- function(model, var1, var2, fieller = FALSE) {
  dof <- df.residual(model)
  beta <- coef(model)
  covar <- vcov(model)
  b1 <- beta[var1]
  b2 <- beta[var2]
  s11 <- covar[var1, var1]
  s22 <- covar[var2, var2]
  s12 <- covar[var1, var2]
  x_min <- min(model$model[var1])
  x_max <- max(model$model[var1])
  xsq <- df[var1]**2
  mode <- "quad"
  t_min <- (b1 + 2 * (b2) * (x_min)) / sqrt(s11 + 4 * (x_min)^2 * s22 + 4 * (x_min) * s12)
  sl_min <- (b1 + 2 * (b2) * (x_min))
  t_max <- (b1 + 2 * (b2) * (x_max)) / sqrt(s11 + 4 * (x_max)^2 * s22 + 4 * (x_max) * s12)
  sl_max <- (b1 + 2 * (b2) * (x_max))
  top <- -(b1) / (2 * b2)
  t_sac <- min(abs(t_min), abs(t_max))
  if (t_min * t_max > 0) {
    t <- NA
    p <- NA
  } else {
    t <- t_sac
    p <- 2 * pt(t_sac, dof, lower.tail = TRUE)
  }
  stars <- ifelse(p < 0.001, "***", ifelse(p < 0.01, "**", ifelse(p <
    0.05, "*", "")))
  breakpoint <- top
  ret <- list(
    statistics = t,
    p.value = p,
    top = top,
    x_min = x_min,
    x_max = x_max
  )
  ret
}
glance_custom.plm <- function(model, ...) {
  ret <- data.frame(
    time_effect = ifelse(
      model$call$effect == "time" | model$call$effect == "twoways",
      "是",
      "否"
    ),
    twoways_effect = ifelse(
      model$call$effect == "twoways",
      "是",
      "否"
    )
  )
  Xs <- (model %>% formula() %>% as.character())[[3]]
  if (str_detect(Xs, "I\\(.+\\^2\\)")) {
    reg.variables <- (formula(model) %>% as.character())[[3]] %>% str_split("\\s*\\+\\s*")
    utest_res <- utest(model, reg.variables[[1]][1], reg.variables[[1]][2])
    stars <- ifelse(utest_res$p.value < 0.001, "***", ifelse(utest_res$p.value <
      0.01, "**", ifelse(utest_res$p.value < 0.05, "*", "")))
    stars <- ifelse(is.na(utest_res$p.value), "", stars)
    ret[["utest_top"]] <- utest_res$top %>%
      round(4) %>%
      as.character() %>%
      paste0(stars)
    ret[["utest_x_min"]] <- utest_res$x_min
    ret[["utest_x_max"]] <- utest_res$x_max
  }
  ret
}


In [3]:
df <- pdata.frame(read.csv(file.path(path.expand("~"), "OneDrive/PhD Dissertation/Data_Code/Data/index_data_regvariables.csv")),
    index = c("Numeric", "Year")
)
df["lnFDIstock_w"] <- as.vector(winsor(df["lnFDIstock"], trim = .01))
df["lnFDIstock_w2"] <- as.vector(winsor(df["lnFDIstock"], trim = .05))
df["s_lnFDIstock_w"] <- as.vector(scale(df["lnFDIstock_w"]))
df["SDI"] <- df["SDI"] * 100
# df['s_lnGNI'] <- as.vector(scale(df['lnGNI']))
df_sum <- df %>% select("SDI", "lnFDIstock_w", "war_intensity")
df_withoutsdi <- df %>% select("lnFDIstock_w", "war_intensity")


ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't subset columns that don't exist.
[31m✖[39m Column `war_intensity` doesn't exist.


In [24]:
model <- plm(SDI ~ lnFDIstock_w2 + I(lnFDIstock_w2^2) + property + war_intensity,
    data = df, model = "within", effect = "time"
)
model %>% modelsummary()

<table class="table" style="width: auto !important; margin-left: auto; margin-right: auto;">
 <thead>
  <tr>
   <th style="text-align:left;">   </th>
   <th style="text-align:center;">  (1) </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> lnFDIstock_w2 </td>
   <td style="text-align:center;"> 17.461 </td>
  </tr>
  <tr>
   <td style="text-align:left;">  </td>
   <td style="text-align:center;"> (6.472) </td>
  </tr>
  <tr>
   <td style="text-align:left;"> I(lnFDIstock_w2^2) </td>
   <td style="text-align:center;"> -0.900 </td>
  </tr>
  <tr>
   <td style="text-align:left;">  </td>
   <td style="text-align:center;"> (0.342) </td>
  </tr>
  <tr>
   <td style="text-align:left;"> property </td>
   <td style="text-align:center;"> 0.398 </td>
  </tr>
  <tr>
   <td style="text-align:left;">  </td>
   <td style="text-align:center;"> (0.058) </td>
  </tr>
  <tr>
   <td style="text-align:left;"> war_intensity </td>
   <td style="text-align:center;"> -1.246 </td>
  </tr>
  <

In [17]:
attributes(model)

$names
 [1] "coefficients" "vcov"         "residuals"    "df.residual"  "formula"     
 [6] "model"        "assign"       "args"         "aliased"      "call"        

$class
[1] "plm"        "panelmodel"

In [21]:
model$model %>% class()

[1] "pdata.frame" "data.frame"

In [8]:
Benchmark <- list(
  "ols" <- lm(SDI ~ lnFDIstock_w, data = df),
  "ols2" <- lm(SDI ~ lnFDIstock_w + property + war_intensity,
    data = df
  ),
  "fixed.time" <- plm(SDI ~ lnFDIstock_w + property + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "fixed.twoways" <- plm(SDI ~ lnFDIstock_w + property + war_intensity,
    data = df, model = "within", effect = "twoways"
  )
)
rows <- tribble(
  ~term, ~ols, ~ols2, ~fixed.time, ~fixed.twoways,
  " ", "OLS", "OLS", "FE_time", "FE_twoways"
)
attr(rows, "position") <- c(1)

Benchmark %>%
  modelsummary(
    fmt = "%.3f",
    stars = SIG,
    estimate = "{estimate}{stars}",
    statistics = "std.error",
    title = "",
    gof_map = gm,
    coef_map = names,
    output = "huxtable",
    add_rows = rows
  )# %>%
  # set_align(everywhere, everywhere, "center") %>%
  # set_top_border(row = 1, col = everywhere) %>%
  # set_top_border(row = -1, col = everywhere, 0) %>%
  # set_bottom_border(row = 2, col = everywhere, 0.4) %>%
  # quick_docx(file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/benchmark.docx"))


,,(1),(2),(3),(4)
,<chr>,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3),(4)
1.1,,OLS,OLS,FE_time,FE_twoways
2,lnFDIstock,0.603,0.567,0.452,0.744
3,,(0.456),(0.351),(0.397),(0.529)
4,property,,0.285***,0.323***,0.044*
5,,,(0.043),(0.054),(0.022)
6,war_intensity,,-2.195***,-2.518***,0.023
7,,,(0.585),(0.782),(0.140)
8,N,64,64,64,64


In [ ]:
Benchmark_robust <- list(
  "average.ols" <- lm(SDI_Average ~ lnFDIstock_w + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "average.fetime" <- plm(SDI_Average ~ lnFDIstock_w + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "average.fetwoways" <- plm(SDI_Average ~ lnFDIstock_w + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "ols.2020" <- lm(SDI ~ lnFDIstock_w + financial + urban + property + tax + war_intensity,
    data = df[df$Year != 2020, ]
  ),
  "fetime.2020" <- plm(SDI ~ lnFDIstock_w + financial + urban + property + tax + war_intensity,
    data = df[df$Year != 2020, ], model = "within", effect = "time"
  ),
  "fetwoways.2020" <- plm(SDI ~ lnFDIstock_w + financial + urban + property + tax + war_intensity,
    data = df[df$Year != 2020, ], model = "within", effect = "twoways"
  ),
  "stockw2.ols" <- lm(SDI ~ lnFDIstock_w2 + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "stockw2.fetime" <- plm(SDI ~ lnFDIstock_w2 + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "stockw2.fetwoways" <- plm(SDI ~ lnFDIstock_w2 + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "lag.ols" <- lm(SDI ~ lnFDIstock_w + lag(financial) + lag(urban) + lag(property) + lag(tax) + lag(war_intensity),
    data = df
  ),
  "lag.fetime" <- plm(SDI ~ lnFDIstock_w + lag(financial) + lag(urban) + lag(property) + lag(tax) + lag(war_intensity),
    data = df, model = "within", effect = "time"
  ),
  "lag.fetwoways" <- plm(SDI ~ lnFDIstock_w + lag(financial) + lag(urban) + lag(property) + lag(tax) + lag(war_intensity),
    data = df, model = "within", effect = "twoways"
  )
)
rows <- tribble(
  ~term, ~average.ols, ~average.fetime, ~average.fetwoways, ~ols.2020, ~fetime.2020, ~fetwoways.2020, ~stockw2.ols, ~stockw2.fetime, ~stockw2.fetwoways, ~lag.ols, ~lag.fetime, ~lag.fetwoways,
  " ", "替换解释变量", "", "", "删除特殊年份", "", "", "5%缩尾处理", "", "", "控制变量滞后一期", "", "",
  " ", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways"
)
attr(rows, "position") <- c(1, 2)
Benchmark_robust <- modelsummary(
  Benchmark_robust,
  fmt = "%.3f",
  stars = SIG,
  estimate = "{estimate}{stars}",
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  add_rows = rows,
  output = "huxtable"
)

attr(rows, "position") <- c(1)
Benchmark_robust <- Benchmark_robust %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0) %>%
  set_bottom_border(row = 2, col = everywhere, 0.4)
quick_docx(Benchmark_robust, file = file.path(Path.HOME, "Downloads/benchmark_robust.docx"))
Benchmark_robust


,,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12)
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12)
1.1,,替换解释变量,,,删除特殊年份,,,5%缩尾处理,,,控制变量滞后一期,,
2,,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways
3,lnFDIstock,-0.003***,-0.004***,-0.014***,-0.433***,-0.577***,-1.419***,-0.401**,-0.548***,-1.443***,-0.405***,-0.543***,-1.493***
4,,(0.001),(0.001),(0.002),(0.152),(0.158),(0.151),(0.156),(0.162),(0.170),(0.148),(0.161),(0.179)
5,financial,0.003***,0.003***,0.002***,0.486***,0.413***,0.177***,0.474***,0.413***,0.179***,,,
6,,(0.001),(0.001),(0.001),(0.087),(0.093),(0.055),(0.085),(0.091),(0.059),,,
7,urban,0.007***,0.007***,0.004**,0.775***,0.761***,0.553***,0.776***,0.782***,0.656***,,,
8,,(0.002),(0.002),(0.002),(0.213),(0.215),(0.167),(0.208),(0.212),(0.172),,,


In [ ]:
# ##内生性检验
# library(estimatr)
# fx <- lm(lnFDIstock_w~lag(lnFDIstock_w,5) + financial + urban + property + tax + war_intensity, data=df)
# lnFDIstock_w_hat <- predict(fx)

# ols<- lm(SDI~lnFDIstock_w + financial + financial + urban + property + tax + war_intensity, data = df)
# tsls<- lm(SDI~lnFDIstock_w_hat + financial + urban + property + tax + war_intensity, data = df)

# iv<- iv_robust(SDI~ lnFDIstock_w + financial + urban + property + tax + war_intensity |lag(lnFDIstock_w,n=5)+ financial + urban + property + tax + war_intensity,
#                data = df)
# summary(iv)


# iv2<- iv_robust(SDI ~lnFDIstock_w + financial + urban + property + tax + war_intensity|lag(lnFDIstock_w,n=5) + lag(lnFDIstock_w,n=10)+ financial + urban + property + tax + war_intensity,
#                data = df,
#                diagnostics= TRUE)
# iv2$diagnostic_first_stage_fstatistic
# scales:: number(iv2$diagnostic_first_stage_fstatistic, accuracy= .01)

# ###we wish this p-value larger than 0.05
# iv2$diagnostic_overid_test

# iv2$diagnostic_endogeneity_test


In [6]:
library(reticulate)
use_condaenv("new-africa")

sns <- import("seaborn")
plt <- import("matplotlib.pyplot")
flatui <- c("#DF7A5E", "#F0C986", "#82B29A", "#3C405B")
common_tick_params <- function (ax) {
    ax$tick_params(
        direction = "in",
        length=4,
        width=1.5,
        colors="black",
        grid_color="black",
        grid_alpha=0.5,
    )
}
sns$set_palette(flatui)

In [70]:
reg <- plm(SDI ~ lnFDIstock + I(lnFDIstock^2) + financial + WGI + urban + tax + war_intensity, data = df, model = "within", effect = "twoways")
new_df <- df
new_df$financial <- mean(new_df$financial)
new_df$WGI <- mean(new_df$WGI)
new_df$urban <- mean(new_df$urban)
new_df$tax <- mean(new_df$tax)
new_df$war_intensity <- mean(new_df$war_intensity)
ax <- sns$scatterplot(data = df, x = "lnFDIstock", y = "SDI")
if (class(reg)[1] == 'plm') {
    fef <- fixef(reg, effect = reg$args$effect)
    y <- predict(reg, new_df) - fef + mean(fef)
} else {
    y <- predict(reg, new_df)
}
sns$lineplot(data = df, x = "lnFDIstock", y = y, color = flatui[3], ax = ax)
ax %>% common_tick_params()
plt$show()

<Axes: xlabel='lnFDIstock', ylabel='SDI'>

In [52]:
fixef(reg, type = "dfirst")

72       120       178       180       204       270       288       404 
-3.990258 -6.302672 -5.232034 -6.703351 -6.269047 -7.421982 -6.271316 -0.070687 
      450       466       562       566       646       686       710       716 
-7.480918 -6.803529 -4.162177  3.063271 -0.133528 -1.270219  4.017376 -3.657541 
      800       854       894 
 5.470828 -0.220706 -3.218515

In [97]:
vars <- str_split(as.character(reg$call$formula[3]), "\\s*\\+\\s*")

In [77]:
summary(reg)

Twoways effects Within Model

Call:
plm(formula = SDI ~ lnFDIstock + I(lnFDIstock^2) + financial + 
    WGI + urban + tax + war_intensity, data = df, effect = "twoways", 
    model = "within")

Balanced Panel: n = 20, T = 16, N = 320

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-3.124275 -0.559797  0.030642  0.582625  4.577376 

Coefficients:
                 Estimate Std. Error t-value  Pr(>|t|)    
lnFDIstock      -2.283573   0.504277 -4.5284 8.826e-06 ***
I(lnFDIstock^2)  0.046388   0.034161  1.3579 0.1755874    
financial        0.190526   0.058189  3.2743 0.0011934 ** 
WGI             -0.534492   0.485177 -1.1016 0.2715692    
urban            0.639767   0.173590  3.6855 0.0002744 ***
tax             16.684458   4.769137  3.4984 0.0005448 ***
war_intensity   -0.101357   0.116686 -0.8686 0.3857960    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    492.51
Residual Sum of Squares: 313.9
R-Squared:      0.36267
Adj. R-Squ

In [ ]:

Benchmark_square <- list(
  "lm.square" <- lm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "fixed.time.square" <- plm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "fixed.twoways.square" <- plm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  )
)
rows <- tribble(
  ~term, ~lm.square, ~fixed.time.square, ~fixed.twoways.square,
  " ",        "OLS",          "FE_time",                  "FE"
)
attr(rows, "position") <- c(1)
Benchmark_square <- modelsummary(
  Benchmark_square,
  fmt = "%.3f",
  stars = SIG,
  estimate = "{estimate}{stars}",
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  add_rows = rows,
  output = "huxtable"
)
# Benchmark_square <- Benchmark_square %>%
#   set_align(everywhere, everywhere, "center") %>%
#   set_top_border(row = 1, col = everywhere) %>%
#   set_top_border(row = -1, col = everywhere, 0) %>%
#   set_bottom_border(row = 2, col = everywhere, 0.4)
# quick_docx(Benchmark_square, file = file.path(Path.HOME, 'OneDrive/PhD Dissertation/Regression Results/benchmark_square.docx'))
Benchmark_square


,,(1),(2),(3)
,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3)
1.1,,OLS,FE_time,FE
2,lnFDIstock,-4.277***,-6.392***,-2.518***
3,,(1.031),(1.090),(0.533)
4,lnFDIstock^2,0.229***,0.341***,0.076**
5,,(0.060),(0.063),(0.035)
6,financial,0.418***,0.284***,0.187***
7,,(0.085),(0.090),(0.057)
8,urban,0.867***,0.908***,0.616***


In [ ]:
library(ggplot2)
# ggplot(data.frame(df$lnFDIstock_w, df$SDI), aes(x=df$lnFDIstock_w, y=df$SDI))+geom_point()


In [ ]:
Benchmark_square_robust <- list(
  "SDIaverage.ols" <- lm(SDI_Average ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "SDIaverage.fetime" <- plm(SDI_Average ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "SDIaverage.fetwoways" <- plm(SDI_Average ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "2020.ols" <- lm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df[df$Year != 2020, ]
  ),
  "2020.fetime" <- plm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df[df$Year != 2020, ], model = "within", effect = "time"
  ),
  "2020.fetwoways" <- plm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + financial + urban + property + tax + war_intensity,
    data = df[df$Year != 2020, ], model = "within", effect = "twoways"
  ),
  "stockw2.ols" <- lm(SDI ~ lnFDIstock_w2 + I(lnFDIstock_w2^2) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "stockw2.fetime" <- plm(SDI ~ lnFDIstock_w2 + I(lnFDIstock_w2^2) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "stockw2.square" <- plm(SDI ~ lnFDIstock_w2 + I(lnFDIstock_w2^2) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "lag.ols" <- lm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + lag(financial) + lag(urban) + lag(property) + lag(tax) + lag(war_intensity),
    data = df
  ),
  "lag.fetime" <- plm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + lag(financial) + lag(urban) + lag(property) + lag(tax) + lag(war_intensity),
    data = df, model = "within", effect = "time"
  ),
  "lag.fetwoways" <- plm(SDI ~ lnFDIstock_w + I(lnFDIstock_w^2) + lag(financial) + lag(urban) + lag(property) + lag(tax) + lag(war_intensity),
    data = df, model = "within", effect = "twoways"
  )
)
Benchmark_square_robust <- modelsummary(
  Benchmark_square_robust,
  fmt = "%.3f",
  stars = SIG,
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  output = "huxtable"
)
Benchmark_square_robust <- Benchmark_square_robust %>%
  insert_row("", "替换解释变量", "", "", "删除特殊年份", "", "", "5%缩尾处理", "", "", "控制变量滞后一期", "", "", after = 1) %>%
  insert_row("", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", after = 2) %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0)
bottom_border(Benchmark_square_robust)[2, ] <- 0.4
quick_docx(Benchmark_square_robust, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/benchmark_square_robust.docx"))
# Benchmark_square_robust


In [ ]:
df["s_lnFDIstock"] <- as.vector(scale(df["lnFDIstock"], scale = FALSE))

In [ ]:
## 经济发展阶段的影响

low_middle <- list(
  "OLS" <- lm(
    SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
      + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "FE_time" <- plm(
    SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
      + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "FE_twoways" <- plm(
    SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
      + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "OLS" <- lm(
    SDI ~ s_lnFDIstock_w + low_income + uppermiddle_income + I(s_lnFDIstock_w * low_income)
      + I(s_lnFDIstock_w * uppermiddle_income) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "FE_time" <- plm(
    SDI ~ s_lnFDIstock_w + low_income + uppermiddle_income + I(s_lnFDIstock_w * low_income)
      + I(s_lnFDIstock_w * uppermiddle_income) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "FE_twoways" <- plm(
    SDI ~ s_lnFDIstock_w + low_income + uppermiddle_income + I(s_lnFDIstock_w * low_income)
      + I(s_lnFDIstock_w * uppermiddle_income) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  )
)
low_middle <- modelsummary(
  low_middle,
  fmt = "%.3f",
  stars = SIG,
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  output = "huxtable"
)
low_middle <- low_middle %>%
  insert_row("", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", after = 1) %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0)
bottom_border(low_middle)[2, ] <- 0.4
# quick_docx(low_middle, file = file.path(Path.HOME, 'OneDrive/PhD Dissertation/Regression Results/low-middle.docx'))
# low_middle


ERROR: Error in eval(predvars, data, env): object 'low_income' not found


In [ ]:
## 经济发展阶段影响的稳健性检验
df2 <- df[df$INCOMEGROUP2005 != 2, ]
low_middle_robust <- list(
   "OLS" <- lm(
      SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df2
   ),
   "FE_time" <- plm(
      SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df2, model = "within", effect = "time"
   ),
   "FE_twoways" <- plm(
      SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df2, model = "within", effect = "twoways"
   ),
   "OLS" <- lm(
      SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df[df$Year != 2020, ]
   ),
   "FE_time" <- plm(
      SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df[df$Year != 2020, ], model = "within", effect = "time"
   ),
   "FE_twoways" <- plm(
      SDI ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df[df$Year != 2020, ], model = "within", effect = "twoways"
   ),
   "OLS" <- lm(
      SDI_Equal ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df
   ),
   "FE_time" <- plm(
      SDI_Equal ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df, model = "within", effect = "time"
   ),
   "FE_twoways" <- plm(
      SDI_Equal ~ s_lnFDIstock_w + low_income + I(s_lnFDIstock_w * low_income)
         + financial + urban + property + tax + war_intensity,
      data = df, model = "within", effect = "twoways"
   )
)
low_middle_robust <- modelsummary(
   low_middle_robust,
   fmt = "%.3f",
   stars = SIG,
   statistics = "std.error",
   title = "",
   gof_map = gm,
   coef_map = names,
   output = "huxtable"
)
low_middle_robust <- low_middle_robust %>%
   insert_row("", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", after = 1) %>%
   set_align(everywhere, everywhere, "center") %>%
   set_top_border(row = 1, col = everywhere) %>%
   set_top_border(row = -1, col = everywhere, 0)
bottom_border(low_middle_robust)[2, ] <- 0.4
quick_docx(low_middle_robust, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/low_middle_robust.docx"))
low_middle_robust


,,(1),(2),(3),(4),(5),(6),(7),(8),(9)
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3),(4),(5),(6),(7),(8),(9)
,,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways
1.1,lnFDIstock,0.018*,0.012,-0.002,0.019***,0.019***,-0.009,0.012***,0.012***,-0.007
2,,(0.011),(0.011),(0.005),(0.005),(0.005),(0.006),(0.004),(0.004),(0.006)
15,low_income,0.032***,0.031***,,0.033***,0.032***,,0.024***,0.023***,
16,,(0.008),(0.008),,(0.006),(0.006),,(0.004),(0.004),
3,lnFDIstock*low_income,-0.031***,-0.029***,-0.016***,-0.031***,-0.036***,-0.013***,-0.021***,-0.024***,-0.013**
4,,(0.011),(0.011),(0.004),(0.006),(0.006),(0.005),(0.004),(0.004),(0.005)
5,financial,0.004***,0.003***,0.002***,0.004***,0.003***,0.002***,0.003***,0.003***,0.002***


In [ ]:
## 石油丰富国家
oilcountry <- list(
  "OLS" <- lm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * oil_country) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "FE_time" <- plm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * oil_country) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "FE_twoways" <- plm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * oil_country) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "OLS" <- lm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * mineral_country) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "FE_time" <- plm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * mineral_country) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "FE_twoways" <- plm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * mineral_country) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  ),
  "OLS" <- lm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * oil_country) + I(s_lnFDIstock_w * mineral_country) + financial + urban + property + tax + war_intensity,
    data = df
  ),
  "FE_time" <- plm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * oil_country) + I(s_lnFDIstock_w * mineral_country) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "time"
  ),
  "FE_twoways" <- plm(SDI ~ s_lnFDIstock_w + I(s_lnFDIstock_w * oil_country) + I(s_lnFDIstock_w * mineral_country) + financial + urban + property + tax + war_intensity,
    data = df, model = "within", effect = "twoways"
  )
)
oilcountry <- modelsummary(
  oilcountry,
  fmt = "%.3f",
  stars = SIG,
  statistics = "std.error",
  title = "",
  gof_map = gm,
  coef_map = names,
  output = "huxtable"
)
oilcountry <- oilcountry %>%
  insert_row("", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", "OLS", "FE_time", "FE_twoways", after = 1) %>%
  set_align(everywhere, everywhere, "center") %>%
  set_top_border(row = 1, col = everywhere) %>%
  set_top_border(row = -1, col = everywhere, 0)
bottom_border(oilcountry)[2, ] <- 0.4
quick_docx(oilcountry, file = file.path(Path.HOME, "OneDrive/PhD Dissertation/Regression Results/oilcountry.docx"))
# oilcountry


,,(1),(2),(3),(4),(5),(6),(7),(8),(9)
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,(1),(2),(3),(4),(5),(6),(7),(8),(9)
,,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways,OLS,FE_time,FE_twoways
1.1,lnFDIstock,-0.009***,-0.012***,-0.023***,-0.001,-0.004,-0.027***,-0.007*,-0.011***,-0.029***
2,,(0.002),(0.003),(0.002),(0.003),(0.004),(0.003),(0.004),(0.004),(0.003)
15,lnFDIstock*oil_country,0.026***,0.029***,0.007,,,,0.024***,0.028***,0.013**
16,,(0.007),(0.007),(0.005),,,,(0.007),(0.007),(0.005)
13,lnFDIstock*mineral_country,,,,-0.010**,-0.010**,0.007***,-0.003,-0.002,0.010***
14,,,,,(0.005),(0.005),(0.003),(0.005),(0.005),(0.003)
3,financial,0.004***,0.003***,0.002***,0.005***,0.004***,0.002***,0.004***,0.003***,0.002***
